# Creating a New SEG-Y

```{article-info}
:author: Altay Sansal
:date: "{sub-ref}`today`"
:read-time: "{sub-ref}`wordcount-minutes` min read"
:class-container: sd-p-0 sd-outline-muted sd-rounded-3 sd-font-weight-light
```

In this tutorial, we create a new SEG-Y file from spec.

Let's start by importing some modules we will be using.

In [ ]:
from segy.factory import SegyFactory
from segy.standards.rev1 import rev1_segy

We will take the default SEG-Y Revision 1 specification.

The `SegyFactory` takes the spec, number of samples, and sample interval as inputs.
By using its creation functions, we can make the encoded (ready to write to disk)
bytes for file headers (text header and binary header).

In [ ]:
SAMPLE_INTERVAL = 4000  # in microseconds
SAMPLES_PER_TRACE = 101

factory = SegyFactory(
    rev1_segy, sample_interval=SAMPLE_INTERVAL, samples_per_trace=SAMPLES_PER_TRACE
)

txt = factory.create_textual_header()
bin_ = factory.create_binary_header()

Let's create 15 traces and populate their values. Headers by default will be populated
by sample rate and number of samples. We will set some fake headers. We will also fill
in the trace samples with `trace_no` + `sample_index`. 

In [ ]:
TRACE_COUNT = 15

headers = factory.create_trace_header_template(size=TRACE_COUNT)
samples = factory.create_trace_sample_template(size=TRACE_COUNT)

for trace_idx in range(TRACE_COUNT):
    headers[trace_idx]["trace_seq_file"] = trace_idx + 1
    headers[trace_idx]["cdp_x"] = 1_000
    headers[trace_idx]["cdp_y"] = 10_000 + trace_idx * 50
    headers[trace_idx]["inline"] = 10
    headers[trace_idx]["crossline"] = 100 + trace_idx

    samples[trace_idx] = range(SAMPLES_PER_TRACE)  # sample index
    samples[trace_idx] += trace_idx  # trace no

Now we can create the encoded binary values for traces (ready to write).

In [ ]:
traces = factory.create_traces(samples=samples, headers=headers)

We can now compose a binary SEG-Y file from pieces.

We create a new `my_segy.sgy` file and write the pieces we built.

In [ ]:
from pathlib import Path

with Path("my_segy.sgy").open(mode="wb") as fp:
    fp.write(txt)
    fp.write(bin_)
    fp.write(traces)

## Opening New SEG-Y

Now we can open it with `SegyFile`. 

Note that our factory correctly populated the revision number in the header so the
spec is automatically inferred!

In [ ]:
from segy.file import SegyFile

file = SegyFile("my_segy.sgy")
print(file.text_header)

In [ ]:
file.binary_header.to_dataframe()

In [ ]:
file.sample[:]

In [ ]:
show_fields = [
    "trace_seq_file",
    "cdp_x",
    "cdp_y",
    "inline",
    "crossline",
]

file.header[:][show_fields].to_dataframe()